In [1]:
%load_ext autoreload
%autoreload 2
%load_ext nb_black
%matplotlib inline
%load_ext tensorboard

import os

# os.environ["CUDA_VISIBLE_DEVICES"] = "-1" #disable gpu


from keras_unet.utils import plot_imgs
from tensorflow import keras
from tensorflow.keras.callbacks import (
    EarlyStopping,
    ModelCheckpoint,
    ReduceLROnPlateau,
    LearningRateScheduler,
    TensorBoard,
)
from tensorflow.keras.layers import Multiply, Input
from tensorflow.keras.models import Model
import tensorflow as tf

import numpy as np
from functools import partial
import segmentation_models as sm
from tensorflow.keras.optimizers import Adam, SGD

physical_devices = tf.config.experimental.list_physical_devices("GPU")
if physical_devices:
    print("There is a gpu avaliable")
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

from myutils.checkpoints import PlotLearning, PredictAndSave
from myutils.models import get_se_model, unet
from myutils.training import (
    get_augmented_data,
    get_val_generator,
    train_val_test_split,
    patch_image,
    get_predictions_from_patches,
    tversky_loss,
)
from myutils.utils import (
    download_dataset,
    load_images,
    process_images,
    prepare_for_pyplot,
)
from myutils.datasets import PatchedDataset, PatchedSequence, Dataset
from myutils.metrics import (
    prediction_report,
    sensitivity_metric,
    specificity_metric,
    g_mean_metric,
)

from myutils.focal_tversky_loss import focal_tversky

from skimage.morphology import dilation, binary_erosion, disk, diamond, area_opening

-----------------------------------------
keras-unet init: TF version is >= 2.0.0 - using `tf.keras` instead of `Keras`
-----------------------------------------
Segmentation Models: using `tf.keras` framework.
There is a gpu avaliable


<IPython.core.display.Javascript object>

In [2]:
seed = 2137

data_dir = '/home/maciej/.keras/datasets/'
input_dir = os.path.join(data_dir, 'images')
target_dir = os.path.join(data_dir, 'manual1')
fov_dir = os.path.join(data_dir, 'mask')

input_img_paths = sorted(
    [
        os.path.join(input_dir, fname)
        for fname in os.listdir(input_dir)
    ]
)
target_img_paths = sorted(
    [
        os.path.join(target_dir, fname)
        for fname in os.listdir(target_dir)
    ]
)
fov_img_paths = sorted(
    [
        os.path.join(fov_dir, fname)
        for fname in os.listdir(fov_dir)
    ]
)

print("Number of samples:", len(input_img_paths))

for input_path, target_path, fov_path in zip(input_img_paths[:10], target_img_paths[:10], fov_img_paths[:10]):
    print(input_path.split('/')[-1].ljust(20), "|", target_path.split('/')[-1].ljust(20), "|", fov_path.split('/')[-1])
print("...")


Number of samples: 45
01_dr.JPG            | 01_dr.tif            | 01_dr_mask.tif
01_g.jpg             | 01_g.tif             | 01_g_mask.tif
01_h.jpg             | 01_h.tif             | 01_h_mask.tif
02_dr.JPG            | 02_dr.tif            | 02_dr_mask.tif
02_g.jpg             | 02_g.tif             | 02_g_mask.tif
02_h.jpg             | 02_h.tif             | 02_h_mask.tif
03_dr.JPG            | 03_dr.tif            | 03_dr_mask.tif
03_g.jpg             | 03_g.tif             | 03_g_mask.tif
03_h.jpg             | 03_h.tif             | 03_h_mask.tif
04_dr.JPG            | 04_dr.tif            | 04_dr_mask.tif
...


<IPython.core.display.Javascript object>

In [3]:
(
    x_train_files,
    x_val_files,
    x_test_files,
    y_train_files,
    y_val_files,
    y_test_files,
    fov_train_files,
    fov_val_files,
    fov_test_files,
) = train_val_test_split(
    input_img_paths,
    target_img_paths,
    fov_img_paths,
    random_state=seed,
    train_size=0.75,
    test_size=0.5,
)

len(x_train)=33
len(x_val)=6
len(x_test)=6


<IPython.core.display.Javascript object>

In [4]:
patch_size = 128 * 5
batch_size = 1

# dataset = PatchedDataset(x_val_directory, y_val_directory, patch_size, patch_size, f'val_{patch_size=}stride={patch_size}')
train_dataset = Dataset("train").from_files(
    x_train_files,
    y_train_files,
    fov_train_files,
    patch_size=patch_size,
    stride=patch_size,
)
val_dataset = Dataset("val").from_files(
    x_val_files, y_val_files, fov_val_files, patch_size=patch_size, stride=patch_size
)
test_dataset = Dataset("test").from_files(x_test_files, y_test_files, fov_test_files)

<IPython.core.display.Javascript object>

In [5]:
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator
import albumentations as A
import math


class AugmentedSequence(Sequence):
    def __init__(
        self,
        x_set,
        y_set,
        mask_set,
        batch_size,
        patch_size,
        random_seed=None,
        augment=True,
        cycles=1,
    ):
        self.x, self.y, self.m = x_set * cycles, y_set * cycles, mask_set * cycles
        self.augment = augment
        self.seed = seed
        self.cycles = cycles
        self.batch_size = batch_size
        self.patch_size = patch_size
        if patch_size is not None:
            self.transform = A.Compose(
                [
                    A.RandomSizedCrop(
                        min_max_height=(patch_size // 2, patch_size * 2),
                        width=patch_size,
                        height=patch_size,
                    ),
                    # A.RandomCrop(width=patch_size, height=patch_size),
                    A.RandomBrightnessContrast(p=0.2),
                    A.VerticalFlip(p=0.5),
                    A.RandomRotate90(p=0.5),
                    A.OneOf(
                        [
                            A.ElasticTransform(
                                p=0.5,
                                alpha=120,
                                sigma=120 * 0.05,
                                alpha_affine=120 * 0.03,
                            ),
                            A.GridDistortion(p=0.5),
                            A.OpticalDistortion(distort_limit=1, shift_limit=0.5, p=1),
                        ],
                        p=0.8,
                    ),
                ],
                additional_targets={"mask0": "mask"},
            )
        else:
            self.transform = A.Compose(
                [
                    A.RandomBrightnessContrast(p=0.2),
                    A.VerticalFlip(p=0.5),
                    A.RandomRotate90(p=0.5),
                    A.OneOf(
                        [
                            A.ElasticTransform(
                                p=0.5,
                                alpha=120,
                                sigma=120 * 0.05,
                                alpha_affine=120 * 0.03,
                            ),
                            A.GridDistortion(p=0.5),
                            A.OpticalDistortion(distort_limit=1, shift_limit=0.5, p=1),
                        ],
                        p=0.1,  # 0.8
                    ),
                ],
                additional_targets={"mask0": "mask"},
            )

        # https://albumentations.ai/docs/examples/example_kaggle_salt/#lets-add-non-rigid-transformations-and-randomsizedcrop

    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size : (idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size : (idx + 1) * self.batch_size]
        batch_m = self.m[idx * self.batch_size : (idx + 1) * self.batch_size]
        X, Y, M = [], [], []
        for x_path, y_path, m_path in zip(batch_x, batch_y, batch_m):
            x, y, m = (
                np.array(load_img(x_path)),
                np.array(load_img(y_path, color_mode="grayscale")),
                np.array(load_img(m_path, color_mode="grayscale")),
            )
            x = x.astype("float32") / 255
            y = np.clip(y.astype("float32"), 0, 1)
            m = np.clip(m.astype("float32"), 0, 1)
            X.append(x)
            Y.append(y)
            M.append(m)

        if self.augment:
            X_trans, Y_trans, M_trans = [], [], []
            for x, y, m in zip(X, Y, M):
                transformed = self.transform(image=x, mask=y, mask0=m)
                X_trans.append(transformed["image"])
                Y_trans.append(transformed["mask"])
                M_trans.append(transformed["mask0"])
        else:
            X_trans, Y_trans, M_trans = X, Y, M

        X_trans, Y_trans, M_trans = (
            np.stack(X_trans, axis=0),
            np.stack(Y_trans, axis=0),
            np.stack(M_trans, axis=0),
        )
        if Y_trans.ndim != 4:
            Y_trans = np.expand_dims(Y_trans, -1)
        if M_trans.ndim != 4:
            M_trans = np.expand_dims(M_trans, -1)
        return (X_trans, M_trans), Y_trans


epochs = 500

train_gen = AugmentedSequence(
    x_train_files,
    y_train_files,
    fov_train_files,
    batch_size,
    patch_size,
    seed,
    cycles=1,
)
"""
train_gen = AugmentedSequence(
    train_dataset.x_paths,
    train_dataset.y_paths,
    train_dataset.m_paths,
    batch_size,
    patch_size,
    seed,
    augment=False,
    cycles=1,
)
"""
# train_gen = PatchedSequence(x_train_files, y_train_files, batch_size, patch_size, seed)

val_gen = AugmentedSequence(
    val_dataset.x_paths,
    val_dataset.y_paths,
    val_dataset.m_paths,
    batch_size,
    patch_size,
    seed,
    False,
)
test_gen = AugmentedSequence(
    test_dataset.x_paths,
    test_dataset.y_paths,
    test_dataset.m_paths,
    1,
    None,
    seed,
    False,
)

<IPython.core.display.Javascript object>

In [6]:
print(train_gen[0][0][0].shape)
print(train_gen[0][0][1].shape)
print(train_gen[0][1].shape)

(1, 640, 640, 3)
(1, 640, 640, 1)
(1, 640, 640, 1)


<IPython.core.display.Javascript object>

In [7]:
# model = sm.Unet(backbone_name='resnet34', input_shape=(patch_size, patch_size, 3), classes=1, encoder_freeze=True)
# model.summary()


def modified_sm_unet(model, patch_size):
    masks = Input(shape=(patch_size, patch_size, 1))
    inputs = model.input
    outputs = model.output
    new_output = Multiply()([outputs, masks])
    new_model = Model(inputs=[inputs, masks], outputs=new_output)

    return new_model

<IPython.core.display.Javascript object>

In [8]:
# %tensorboard --logdir f"./logs-{chp_tmp}/scalars"

<IPython.core.display.Javascript object>

In [9]:
import datetime


def get_ftime():
    return datetime.datetime.today().strftime("%Y-%B-%d_%H:%M:%S")


def get_model_checkpoint():
    return f"model_checkpoint-{get_ftime()}.h5"


model_checkpoint = get_model_checkpoint()

print(model_checkpoint)

from tensorflow.keras import backend as K


class LRTensorBoard(TensorBoard):
    def __init__(
        self, log_dir, **kwargs
    ):  # add other arguments to __init__ if you need
        super().__init__(log_dir=log_dir, **kwargs)

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        logs.update({"lr": K.eval(self.model.optimizer.lr)})
        super().on_epoch_end(epoch, logs)


def scheduler(epoch, lr):
    if epoch < 200:
        return 1e-4
    else:
        return lr * tf.math.exp(-0.1)


chp_tmp = "resnet50_augmented_adam_bce_jaccard_{}.h5"
best_checkpoint = chp_tmp.format("best")
best_checkpoint_g_mean = chp_tmp.format("best_g_mean")
last_checkpoint = chp_tmp.format("last")
print(f"{best_checkpoint=}")
print(f"{best_checkpoint_g_mean=}")
print(f"{last_checkpoint=}")

model_checkpoint-2021-May-24_16:40:20.h5
best_checkpoint='resnet50_augmented_adam_bce_jaccard_best.h5'
best_checkpoint_g_mean='resnet50_augmented_adam_bce_jaccard_best_g_mean.h5'
last_checkpoint='resnet50_augmented_adam_bce_jaccard_last.h5'


<IPython.core.display.Javascript object>

In [10]:
keras.backend.clear_session()

optimizer = Adam(lr=1e-4)
# optimizer = SGD(lr=1e-4)

# loss = sm.losses.bce_jaccard_loss
loss = tversky_loss
# loss = sm.losses.dice_loss

metrics = [
    sm.metrics.IOUScore(threshold=0.5),
    sm.metrics.FScore(threshold=0.5),
    tf.keras.metrics.BinaryAccuracy(),
    sensitivity_metric,
    specificity_metric,
    g_mean_metric,
]

callbacks = [
    ModelCheckpoint(
        best_checkpoint, monitor="val_loss", save_best_only=True, verbose=1
    ),
    ModelCheckpoint(
        best_checkpoint_g_mean,
        monitor="val_g_mean_metric",
        save_best_only=True,
        verbose=1,
    ),
    ModelCheckpoint(last_checkpoint, save_best_only=False, verbose=0),
    ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=50, min_lr=1e-6, verbose=1
    ),
    LRTensorBoard(log_dir=f"./logs"),
]

<IPython.core.display.Javascript object>

In [11]:
# model = unet(input_shape=(patch_size, patch_size, 3), classes=1)

model = sm.Unet(
    "resnet50", (patch_size, patch_size, 3), activation="sigmoid", encoder_freeze=True
)

model = modified_sm_unet(model, patch_size)
model.summary()

model.compile(optimizer, loss=loss, metrics=metrics)
model.fit(train_gen, epochs=1, verbose=1)

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               [(None, 640, 640, 3) 0                                            
__________________________________________________________________________________________________
bn_data (BatchNormalization)    (None, 640, 640, 3)  9           data[0][0]                       
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 646, 646, 3)  0           bn_data[0][0]                    
__________________________________________________________________________________________________
conv0 (Conv2D)                  (None, 320, 320, 64) 9408        zero_padding2d[0][0]             
____________________________________________________________________________________________

33/33 [==============================] - 18s 263ms/step - loss: 0.7311 - iou_score: 0.1257 - f1-score: 0.2181 - binary_accuracy: 0.5285 - sensitivity_metric: 0.7114 - specificity_metric: 0.5083 - g_mean_metric: 0.5999


<IPython.core.display.Javascript object>

In [12]:
try:
    model.load_weights(model_checkpoint)
except:
    pass
for layer in model.layers:
    layer.trainable = True

# model.compile(optimizer, loss=loss, metrics=metrics)
history = model.fit(
    train_gen, epochs=epochs, validation_data=val_gen, callbacks=callbacks, verbose=1
)

Epoch 1/500
33/33 [==============================] - 12s 367ms/step - loss: 0.6022 - iou_score: 0.2416 - f1-score: 0.3758 - binary_accuracy: 0.7390 - sensitivity_metric: 0.8589 - specificity_metric: 0.7284 - g_mean_metric: 0.7898 - val_loss: 0.7740 - val_iou_score: 0.0867 - val_f1-score: 0.1577 - val_binary_accuracy: 0.2273 - val_sensitivity_metric: 0.9994 - val_specificity_metric: 0.1575 - val_g_mean_metric: 0.2579

Epoch 00001: val_loss improved from inf to 0.77398, saving model to resnet50_augmented_adam_bce_jaccard_best.h5

Epoch 00001: val_g_mean_metric improved from inf to 0.25793, saving model to resnet50_augmented_adam_bce_jaccard_best_g_mean.h5
Epoch 2/500
33/33 [==============================] - 12s 356ms/step - loss: 0.5555 - iou_score: 0.3060 - f1-score: 0.4557 - binary_accuracy: 0.8288 - sensitivity_metric: 0.8630 - specificity_metric: 0.8265 - g_mean_metric: 0.8442 - val_loss: 0.7669 - val_iou_score: 0.0864 - val_f1-score: 0.1572 - val_binary_accuracy: 0.2406 - val_sensit


Epoch 00015: val_loss did not improve from 0.76508

Epoch 00015: val_g_mean_metric did not improve from 0.07635
Epoch 16/500
33/33 [==============================] - 11s 331ms/step - loss: 0.3411 - iou_score: 0.4964 - f1-score: 0.6461 - binary_accuracy: 0.9320 - sensitivity_metric: 0.7999 - specificity_metric: 0.9464 - g_mean_metric: 0.8693 - val_loss: 0.9403 - val_iou_score: 0.0209 - val_f1-score: 0.0405 - val_binary_accuracy: 0.9066 - val_sensitivity_metric: 0.0277 - val_specificity_metric: 0.9787 - val_g_mean_metric: 0.1528

Epoch 00016: val_loss did not improve from 0.76508

Epoch 00016: val_g_mean_metric did not improve from 0.07635
Epoch 17/500
33/33 [==============================] - 11s 330ms/step - loss: 0.3213 - iou_score: 0.5184 - f1-score: 0.6673 - binary_accuracy: 0.9349 - sensitivity_metric: 0.7949 - specificity_metric: 0.9506 - g_mean_metric: 0.8680 - val_loss: 0.7700 - val_iou_score: 0.0867 - val_f1-score: 0.1578 - val_binary_accuracy: 0.2563 - val_sensitivity_metric: 

KeyboardInterrupt: 

<IPython.core.display.Javascript object>

In [ ]:
model.save_weights(model_checkpoint)

In [ ]:
from tensorflow.keras.preprocessing.image import load_img
import matplotlib.pyplot as plt
model.load_weights(best_checkpoint)
(x_to_predict, m), y_to_predict = val_gen[4]
pred = model.predict([x_to_predict, m])
print(x_to_predict.shape, y_to_predict.shape, pred.shape)
plot_imgs(prepare_for_pyplot(x_to_predict), y_to_predict, pred)


In [ ]:
model.load_weights(best_checkpoint)
(x_to_predict, m), y_to_predict = val_gen[5]
pred = model.predict([x_to_predict, m])
print(x_to_predict.shape, y_to_predict.shape, pred.shape)
plot_imgs(prepare_for_pyplot(x_to_predict), y_to_predict, pred)

In [ ]:
from PIL import Image

path_x = "/home/maciej/.keras/datasets/images/05_g.jpg"
path_y = "/home/maciej/.keras/datasets/manual1/05_g.tif"
path_m = "/home/maciej/.keras/datasets/mask/05_g_mask.tif"

# model = sm.Unet(backbone_name='resnet34', input_shape=(patch_size, patch_size, 3), classes=1, encoder_freeze=False)
# model = modified_sm_unet(model, patch_size)
# model.load_weights('augmented_adam_focal_best.h5')
# model.load_weights('resnet_34_augmented_adam_tversky_best-Copy1.h5')

model.load_weights("resnet_34_augmented_adam1e4_tversky_best-Copy1.h5")
x_whole = np.array(load_img(path_x))
y_whole = np.array(load_img(path_y, color_mode="grayscale"))
m = np.array(load_img(path_m, color_mode="grayscale"))
# m = np.ones_like(m)
x_whole = x_whole.astype("float32") / 255
y_whole = np.clip(y_whole, 0, 1).astype("float32")
m = np.clip(m, 0, 1).astype("float32")

# print(mask.shape, m.shape)
mask = ~binary_erosion(m, disk(2))
m[mask] = 0

x_whole, y_whole, m = (
    np.expand_dims(x_whole, 0),
    np.expand_dims(y_whole, [0, -1]),
    np.expand_dims(m, [0, -1]),
)


pred = get_predictions_from_patches(
    model, x_whole, m, patch_size, patch_size, threshold=None
)
print(x_whole.shape, y_whole.shape, m.shape, pred.shape)
plot_imgs(prepare_for_pyplot(x_whole), y_whole, pred)

print(np.squeeze(pred[0], -1).shape)
img = prepare_for_pyplot(np.squeeze(pred[0], -1))
im = Image.fromarray(img * 255)
# im.save("pred.jpg")
im.show()

In [ ]:
from PIL import Image

threshold = 0.14

save_dir = f"predictions_na_5_thr={threshold}"
try:
    os.mkdir(save_dir)
except:
    pass


for x_test_file, y_test_file, m_file in zip(x_test_files, y_test_files, fov_test_files):
    file_name = x_test_file.split("/")[-1].split(".")[0]

    x_whole = np.array(load_img(x_test_file))
    y_whole = np.array(load_img(y_test_file, color_mode="grayscale"))
    m = np.array(load_img(m_file, color_mode="grayscale"))

    x_whole = x_whole.astype("float32") / 255
    y_whole = np.clip(y_whole, 0, 1).astype("float32")
    m = np.clip(m, 0, 1).astype("float32")

    x_whole, y_whole, m = (
        np.expand_dims(x_whole, 0),
        np.expand_dims(y_whole, [0, -1]),
        np.expand_dims(m, [0, -1]),
    )

    pred = get_predictions_from_patches(
        model, x_whole, m, patch_size, patch_size, threshold=threshold
    )
    pred = np.squeeze(pred[0], -1)
    im = Image.fromarray(np.uint8(pred * 255))
    im.save(f"{save_dir}/{file_name}-p.png")

    x_whole = x_whole[0]
    x_whole[pred == 1] = np.array([0, 1, 0])
    im = Image.fromarray(np.uint8(x_whole * 255))
    im.save(f"{save_dir}/{file_name}-o.png")

    y_whole = np.squeeze(y_whole, (0, 3))
    im = Image.fromarray(np.uint8(y_whole * 255))
    im.save(f"{save_dir}/{file_name}-m.png")

In [ ]:
from myutils.metrics import prediction_report
import pprint
import pandas as pd

def find_best_threshold(model, train_seq, patch_size, stride, target_metric='accuracy', steps=10):
    def helper(left_thr, right_thr, left_metric, right_metric, steps):
        thr = (left_thr + right_thr) / 2
        report = prediction_report(model, train_seq, patch_size=patch_size, stride=patch_size, threshold=thr)
        metric = report[1][target_metric]
        if steps == 0:
            return max((left_metric, left_thr), (right_metric, right_thr), (metric, thr))
        else:
            if metric > left_metric:
                return helper(thr, right_thr, metric, right_metric, steps-1)
            else:
                return helper(left_thr, left_metric, thr, metric, steps-1)
    report = prediction_report(model, train_seq, patch_size=patch_size, stride=patch_size, threshold=0.05)
    left_met = report[1][target_metric]
    report = prediction_report(model, train_seq, patch_size=patch_size, stride=patch_size, threshold=1-0.05)
    right_met = report[1][target_metric]
    
    return helper(0.05, 1-0.05, left_met, right_met, steps)

def linear_search(model, train_seq, patch_size, stride, steps=10):
    thresholds = np.linspace(0.05, 1-0.05, steps)
    for thr in thresholds:
        report = prediction_report(model, train_seq, patch_size=patch_size, stride=patch_size, threshold=thr)
        print(f"{thr=}")
        pd.options.display.float_format = "{:,.2f}".format
        df = pd.DataFrame(report)
        print(df, "\n")

metric = 'accuracy'
# best_thr = find_best_threshold(model, train_gen, patch_size, patch_size, metric, 10)
linear_search(model, val_gen, patch_size, patch_size, 20)
print(f"Best threshold={best_thr[1]} gives {metric}={best_thr[0]} on training set (without augmentation)")


In [ ]:
import pandas as pd

# model.load_weights('augmented_adam_dice_best-Copy1.h5')
model.load_weights(best_checkpoint)
# report = prediction_report(model, test_gen, patch_size=patch_size, stride=patch_size, threshold=best_thr[1])
report = prediction_report(
    model, test_gen, patch_size=patch_size, stride=patch_size, threshold=0.334
)

# pprint.pprint(report)

pd.options.display.float_format = "{:,.2f}".format
df = pd.DataFrame(report)
print(df)

In [ ]:
import pandas as pd

# model.load_weights('augmented_adam_dice_best-Copy1.h5')
# model.load_weights(best_checkpoint)
# report = prediction_report(model, test_gen, patch_size=patch_size, stride=patch_size, threshold=best_thr[1])
report = prediction_report(
    model, test_gen, patch_size=patch_size, stride=patch_size, threshold=0.5
)

# pprint.pprint(report)

pd.options.display.float_format = "{:,.2f}".format
df = pd.DataFrame(report)
print(df)

In [ ]:
"""
Using fov

sgd bce unet
thr=0.25
support     42,236,884.00 4,216,591.00
accuracy             0.96         0.96
precision            0.98         0.78
recall               0.98         0.83
f1                   0.98         0.81
sensitivity          0.98         0.83
specifity            0.83         0.98 

thr=0.35
                        0            1
support     42,236,884.00 4,216,591.00
accuracy             0.97         0.97
precision            0.98         0.83
recall               0.98         0.78
f1                   0.98         0.80
sensitivity          0.98         0.78
specifity            0.78         0.98 




Adam(1e-4) TverskyLoss(a=0.3, b=0.7) resnet34 patch_size=1024  batch_size=2 200 epochs
resnet_34_augmented_adam1e4_tversky_*-Copy1
                        0            1
support     37,538,905.00 3,943,117.00
accuracy             0.96         0.96
precision            0.99         0.72
recall               0.96         0.87
f1                   0.98         0.79
sensitivity          0.96         0.87
specifity            0.87         0.96

Adam(4e-5) TverskyLoss(a=0.3, b=0.7) resnet34 patch_size=1024  batch_size=2 200 epochs
resnet_34_augmented_adam_tversky_*-Copy1   ()
                        0            1
support     37,538,905.00 3,943,117.00
accuracy             0.96         0.96
precision            0.99         0.73
recall               0.97         0.86
f1                   0.98         0.79
sensitivity          0.97         0.86
specifity            0.86         0.97

Adam(1e-5) DiceLoss([0.1, 1]) resnet34 patch_size=512  batch_size=1 500 epochs
resnet_34_augmented_adam_dice_class_weights
                        0            1
support     37,538,905.00 3,943,117.00
accuracy             0.95         0.95
precision            0.97         0.78
recall               0.98         0.73
f1                   0.98         0.75
sensitivity          0.98         0.73
specifity            0.73         0.98

Adam(4e-5) DiceLoss() resnet34 patch_size=1024 batch_size=2 epochs=250
                        0            1
support     37,538,905.00 3,943,117.00
accuracy             0.96         0.96
precision            0.98         0.81
recall               0.98         0.78
f1                   0.98         0.80
sensitivity          0.98         0.78
specifity            0.78         0.98

----------------------

Adam(8e-5) DiceLoss()+JaccardLoss() resnet34 patch_size=512 batch_size=8 epochs=300
                        0            1
support     30,418,540.00 2,322,836.00
accuracy             0.97         0.97
precision            0.98         0.82
recall               0.99         0.69
f1                   0.98         0.75
sensitivity          0.99         0.69
specifity            0.69         0.99

Adam(1e-4) DiceLoss() resnet34 patch_size=1024 batch_size=2 epochs=500
                        0            1
support     53,264,590.00 4,032,818.00
accuracy             0.97         0.97
precision            0.98         0.79
recall               0.98         0.80
f1                   0.98         0.79
sensitivity          0.98         0.80
specifity            0.80         0.98

Adam(1e-5) DiceLoss() resnet34 patch_size=1024 batch_size=2 epochs=500
                        0            1
support     53,264,590.00 4,032,818.00
accuracy             0.97         0.97
precision            0.99         0.78
recall               0.98         0.81
f1                   0.98         0.79
sensitivity          0.98         0.81
specifity            0.81         0.98
"""